In [ ]:
#Importo le librerie 
import pandas as pd

In [ ]:
#Ottengo il dataframe usando le API del sito: https://bechdeltest.com/
#Il metodo usato mi fornisce 5 colonne
df = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies')
df.to_csv('Bechdel_originale.csv') #salvo il dataframe originale

#Elimino gli anni dei film che non mi servono
df.drop(df.loc[df['year']<2019].index, inplace=True)
df.drop(df.loc[df['year']>2021].index, inplace=True)

#Ordino l'indice esterno
df = df.reset_index(drop=True)

#Salvo il dataframe in un csv
df.to_csv('Bechdel.csv')

In [ ]:
#Uso un altro metodo fornito dalle API per avere più informazioni su ogni film
count = 1
#Creo un dataframe vuoto
df_new = pd.DataFrame(columns=['title', 'year', 'rating', 'dubious', 'imdbid', 'id', 'submitterid', 'date', 'visible'])

#Chiamo i film dall'ultimo al primo film, nel caso in cui il sito web si arresti in modo anomalo
for imdbid in df['imdbid'][-1::-1]:
    series = pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(imdbid), typ='series')
    df_new = df_new.append(series, ignore_index=True)
    # ogni 10 film li salviamo in un csv, 
    # così non si perde nulla nel caso ci fosse un errore
    if count % 10 == 0:
        df_new.to_csv('Bechdel_detailed_reversed.csv')
    count += 1
df_new.to_csv('Bechdel_detailed_reversed.csv')

#Ribalto l'ordine delle righe
df_new = df_new.iloc[::-1].reset_index(drop=True)
df_new.to_csv('Bechdel_detailed_originale.csv')
#Ora i film in Bechdel_detailed.csv dovrebbero corrispondere all'ordine di quelli in Bechdel.csv

In [ ]:
bechdel_detailed_df = pd.read_csv('Bechdel_detailed_originale.csv', index_col=0) 
bechdel_detailed_df.info() #Guardo le informazioni del dataset

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        493 non-null    object 
 1   year         493 non-null    float64
 2   rating       493 non-null    float64
 3   dubious      493 non-null    float64
 4   imdbid       493 non-null    float64
 5   id           493 non-null    float64
 6   submitterid  493 non-null    float64
 7   date         493 non-null    object 
 8   visible      493 non-null    float64
 9   description  2 non-null      object 
 10  status       2 non-null      float64
 11  version      2 non-null      object 
dtypes: float64(8), object(4)
memory usage: 50.3+ KB


In [ ]:
#Controllo se ci sono valori nulli e risolvo il problema
bechdel_detailed_df[bechdel_detailed_df['title'].isna()]

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,description,status,version
236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1
477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1


In [ ]:
#Controllo nel primo dataframe quei codici a quali film corrispondono e che 
#problema possono avere
df.iloc[[236, 477]] #non hanno imdbid quindi danno Nan nell'altro

,rating,imdbid,year,id,title
236,3,,2019,10029,Decedants 3
477,3,,2021,10155,Cruella


In [ ]:
#Modifico Bechdel.csv inserendo gli imdbid mancanti
bechdel_df = pd.read_csv('Bechdel.csv', index_col=0)
bechdel_df.at[236, 'imdbid'] = 8022928
bechdel_df.at[477, 'imdbid'] = 3228774
bechdel_df['imdbid'] = bechdel_df['imdbid'].astype('int')
bechdel_df.to_csv('Bechdel_finale.csv')

#Modifico Bechdel_detailed.csv
bechdel_detailed_df = pd.read_csv('./Bechdel_detailed_originale.csv', index_col=0)
for imdbid, index in zip([8022928, 3228774], [236, 477]):
    series = pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(imdbid), typ='series')
    bechdel_detailed_df.loc[index] = series

#Levo le colonne inutili
bechdel_detailed_df.drop(['description', 'status', 'version', 'visible', 'submitterid', 'id', 'date'], axis=1, inplace=True)

#Modifico il tipo di dato delle colonne dove ci sono dei numeri. 
#Li salvo tutti come interi
bechdel_detailed_df['year'] = bechdel_detailed_df['year'].astype('int')
bechdel_detailed_df['rating'] = bechdel_detailed_df['rating'].astype('int')
bechdel_detailed_df['dubious'] = bechdel_detailed_df['dubious'].astype('int')
bechdel_detailed_df['imdbid'] = bechdel_detailed_df['imdbid'].astype('int')
bechdel_detailed_df.to_csv('Bechdel_detailed.csv')

In [ ]:
#Voglio trovare se ci sono delle righe duplicate
id = bechdel_detailed_df['imdbid'].unique()
for codice in id:
  if len(bechdel_detailed_df[bechdel_detailed_df['imdbid'] == codice]) > 1:
    print(codice)

2386490
8022928
3228774


In [ ]:
bechdel_detailed_df[bechdel_detailed_df['imdbid'] == 2386490]
#Tengo la riga 220 perché ho letto i commenti sul sito ed è più attendibile

,title,year,rating,dubious,imdbid
1,How to Train Your Dragon: The Hidden World,2019,2,0,2386490
220,How to Train Your Dragon: The Hidden World,2019,3,0,2386490


In [ ]:
bechdel_detailed_df[bechdel_detailed_df['imdbid'] == 8022928]
#tengo la riga 65 perché sono uguali

,title,year,rating,dubious,imdbid
65,Descendants 3,2019,3,0,8022928
236,Descendants 3,2019,3,0,8022928


In [ ]:
bechdel_detailed_df[bechdel_detailed_df['imdbid'] == 3228774]
#tengo la riga 407 perché sono uguali

,title,year,rating,dubious,imdbid
407,Cruella,2021,3,0,3228774
477,Cruella,2021,3,0,3228774


In [ ]:
#Levo le righe duplicate
bechdel_detailed_df.drop(1, inplace=True)
bechdel_detailed_df.drop(236, inplace=True)
bechdel_detailed_df.drop(477, inplace=True)

#Sistemo l'indice esterno
bechdel_detailed_df = bechdel_detailed_df.reset_index(drop=True)
bechdel_detailed_df.to_csv('Bechdel_detailed_finale.csv')

In [ ]:
#Questi sono il totale di quelli con la colonna dubbia = 1
bechdel_detailed_df[bechdel_detailed_df['dubious'] == 1].sum() #sono 49

title      Wandering Earth (Liu Lang Di Qiu), TheGully Bo...
year                                                   98979
rating                                                   125
dubious                                                   49
imdbid                                             352156761
dtype: object

# ID title imdb

Aggiungo "tt" ad ogni id del title del dataset per il test di Bechdel e lo trasformo in stringa.

In [ ]:
Bechdel_df = pd.read_csv('Bechdel_detailed_finale.csv', index_col=[0]) 

In [ ]:
Bechdel_df.columns 

Index(['title', 'year', 'rating', 'dubious', 'imdbid'], dtype='object')

In [ ]:
print(Bechdel_df['imdbid'].dtypes)

int64


In [ ]:
Bechdel_df['imdbid'] = Bechdel_df['imdbid'].astype(str) 

In [ ]:
print(Bechdel_df['imdbid'].dtypes) 

object


In [ ]:
Bechdel_df['imdbid'] = 'tt' + Bechdel_df['imdbid'] 

In [ ]:
Bechdel_df['imdbid']

0       tt8179388
1       tt9358120
2       tt6811018
3       tt4139588
4       tt5316540
          ...    
487    tt10370710
488     tt4995540
489    tt11127680
490    tt10293406
491    tt11389748
Name: imdbid, Length: 492, dtype: object

In [ ]:
Bechdel_df.columns 

Index(['title', 'year', 'rating', 'dubious', 'imdbid'], dtype='object')

In [ ]:
Bechdel_df.head()

,title,year,rating,dubious,imdbid
0,Rim of the World,2019,0,0,tt8179388
1,We Are Little Zombies,2019,2,0,tt9358120
2,"Kid Who Would Be King, The",2019,1,0,tt6811018
3,Polar,2019,2,0,tt4139588
4,Close,2019,3,0,tt5316540


In [ ]:
Bechdel_df.to_csv('Bechdel_dataset.csv', index=False)

# Controllo problemi derivanti da html

In [ ]:
df_Bechdel = pd.read_csv('Bechdel_dataset.csv')

In [ ]:
for titolo in df_Bechdel['title']:
  if titolo.find('Retreat') != -1:
    print(titolo)

Retreat, The
Retreat, The


In [ ]:
df_Bechdel[df_Bechdel['title'] == 'Retreat, The']
#Siamo andate a verificare se fosse un caso di duplicazione della riga, invece, 
#esistono due film con lo stesso titolo prodotti in due anni diversi e che non 
#sono correlati tra loro.

,title,year,rating,dubious,imdbid
351,"Retreat, The",2020,0,0,tt12198524
472,"Retreat, The",2021,3,0,tt11392272
